In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

def export_DfExcel (df, fileNme):
    try:
        df.to_excel(fileNme)
        print("Exported")
    except:
        print("Failure")


/kaggle/input/formula-1-world-championship-1950-2020/races.csv
/kaggle/input/formula-1-world-championship-1950-2020/constructor_results.csv
/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv
/kaggle/input/formula-1-world-championship-1950-2020/constructors.csv
/kaggle/input/formula-1-world-championship-1950-2020/lap_times.csv
/kaggle/input/formula-1-world-championship-1950-2020/status.csv
/kaggle/input/formula-1-world-championship-1950-2020/driver_standings.csv
/kaggle/input/formula-1-world-championship-1950-2020/seasons.csv
/kaggle/input/formula-1-world-championship-1950-2020/pit_stops.csv
/kaggle/input/formula-1-world-championship-1950-2020/sprint_results.csv
/kaggle/input/formula-1-world-championship-1950-2020/constructor_standings.csv
/kaggle/input/formula-1-world-championship-1950-2020/results.csv
/kaggle/input/formula-1-world-championship-1950-2020/circuits.csv
/kaggle/input/formula-1-world-championship-1950-2020/qualifying.csv


## Import

-  Constructors
-  races
-  results
-  status

In [45]:
results_df = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/results.csv',
                       index_col = 'resultId')

races_df = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/races.csv',
                       index_col = 'raceId')

constructors_df = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructors.csv',
                             index_col = 'constructorId')

## Merge df to analytical set



In [46]:

rc_df = pd.merge(results_df,constructors_df, on = 'constructorId' )
rc_df.head(5)

,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,...,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,constructorRef,name,nationality,url
0,18,1,1,22,1,1,1,1,10.0,58,...,5690616,39,2,1:27.452,218.300,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,18,2,2,3,5,2,2,2,8.0,58,...,5696094,41,3,1:27.739,217.586,1,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,18,3,3,7,7,3,3,3,6.0,58,...,5698779,41,5,1:28.090,216.719,1,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,18,4,4,5,11,4,4,4,5.0,58,...,5707797,58,7,1:28.603,215.464,1,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,18,5,1,23,3,5,5,5,4.0,58,...,5708630,43,1,1:27.418,218.385,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


## Creating a Race Name with Year - Race

## add Race Name to df

In [47]:
# rename constructr name to construtor name
rc_df.rename(columns={'name': 'ConstructorNME'}, inplace=True)
rc_df.drop(columns=['positionText', 'constructorRef'], inplace=True)

In [48]:
rc_df = pd.merge(rc_df, races_df[['name', 'year']], on = 'raceId')
#rc_df.rename(columns={'name': 'RaceNme'}, inplace=True)


## Combine Race and Year

In [49]:
rc_df["race"] = rc_df['year'].astype(str) +"-"+ rc_df["name"]
# rc_df.drop(columns=['year', 'name'], inplace=True)

### Export the dataframe as a CSV for future use

### Group by Category, sum and sort

In [50]:
# Group by 'Category', sum 'Value', and sort
TeamsByPoints = rc_df.groupby('ConstructorNME')['points'].sum().sort_values(ascending=False)
print(TeamsByPoints)

ConstructorNME
Ferrari      11091.27
Mercedes      7730.64
Red Bull      7673.00
McLaren       7022.50
Williams      3641.00
               ...   
Emeryson         0.00
McGuire          0.00
Euro Brun        0.00
Ewing            0.00
AFM              0.00
Name: points, Length: 211, dtype: float64


In [51]:
fileNme = 'TraditionalPoints.xlsx'
df = TeamsByPoints
export_DfExcel (df, fileNme)

Exported


## Scoring System Recency Bias

The combination of inflating the point scoring system and addition of more opportunities to score points creates a recency bias.  In 2010 the F1 changed the scoring system a win's points changed from 10 to 25 points.  This bias was exasperated in 2019 by adding a point for the fastest lap and in 2021 with the addition of sprint races.  In 1950 a the winning driver of a Formula one weekend could walk away with 9 points; 8 for winning the race plus 1 for fastest lap.  In 2024 the most a driver could win in a weekend is  34; 8 for winning the sprint race



1950 to 1959  top five finishers and the fastest lap setter were awarded points
1960 to 2002  top six finishers but with no points for the fastest lap 
2003 to 2009  top eight finishers, no points for fastest lap
2010 to 2018 top ten finishers
2019 to preesnt - 1 point for the fastest lap reintroduced
2021 to present  Sprint races were added with points awarded for the first eight finishers

### Consistent Scoring
As each race has had at least 10 finishers, rescoring the races 10 - 1

https://www.planetf1.com/news/f1-points-all-time-rankings
https://en.wikipedia.org/wiki/List_of_Formula_One_World_Championship_points_scoring_systems
https://www.motorsport.com/f1/news/f1-to-drop-point-for-fastest-lap-after-2024-season/10663520/#:~:text=From%20next%20year%2C%20F1%20will,something%20extra%20to%20race%20for.

### insert a field (column) to add a standardized score

In [52]:
rc_df.insert(8, 'StandardPoints', 0)

#rc_df.insert(loc=8, column='StandardPoints', value=pd.Series(dtype='int')) 
rc_df.head(5)

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,raceId,driverId,constructorId,number,grid,position,positionOrder,points,StandardPoints,laps,...,rank,fastestLapTime,fastestLapSpeed,statusId,ConstructorNME,nationality,url,name,year,race
0,18,1,1,22,1,1,1,10.0,NaN,58,...,2,1:27.452,218.300,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Australian Grand Prix,2008,2008-Australian Grand Prix
1,18,2,2,3,5,2,2,8.0,NaN,58,...,3,1:27.739,217.586,1,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,Australian Grand Prix,2008,2008-Australian Grand Prix
2,18,3,3,7,7,3,3,6.0,NaN,58,...,5,1:28.090,216.719,1,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Australian Grand Prix,2008,2008-Australian Grand Prix
3,18,4,4,5,11,4,4,5.0,NaN,58,...,7,1:28.603,215.464,1,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,Australian Grand Prix,2008,2008-Australian Grand Prix
4,18,5,1,23,3,5,5,4.0,NaN,58,...,1,1:27.418,218.385,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Australian Grand Prix,2008,2008-Australian Grand Prix


### Create a function to rescore the the finish

In [53]:
def rescore (pos):
    resc = {
        1: 10,
        2: 9,
        3: 8,
        4: 7,
        5: 6,
        6: 5,
        7: 4,
        8: 3,
        9: 2,
        10: 1
    }
    return resc.get(pos, 0)

In [54]:
rescore(10)

1

In [55]:
rc_df['StandardPoints'] = rc_df['positionOrder'].apply(rescore)

In [56]:
rc_df.head(10)

,raceId,driverId,constructorId,number,grid,position,positionOrder,points,StandardPoints,laps,...,rank,fastestLapTime,fastestLapSpeed,statusId,ConstructorNME,nationality,url,name,year,race
0,18,1,1,22,1,1,1,10.0,10,58,...,2,1:27.452,218.300,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Australian Grand Prix,2008,2008-Australian Grand Prix
1,18,2,2,3,5,2,2,8.0,9,58,...,3,1:27.739,217.586,1,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,Australian Grand Prix,2008,2008-Australian Grand Prix
2,18,3,3,7,7,3,3,6.0,8,58,...,5,1:28.090,216.719,1,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Australian Grand Prix,2008,2008-Australian Grand Prix
3,18,4,4,5,11,4,4,5.0,7,58,...,7,1:28.603,215.464,1,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,Australian Grand Prix,2008,2008-Australian Grand Prix
4,18,5,1,23,3,5,5,4.0,6,58,...,1,1:27.418,218.385,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Australian Grand Prix,2008,2008-Australian Grand Prix
5,18,6,3,8,13,6,6,3.0,5,57,...,14,1:29.639,212.974,11,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Australian Grand Prix,2008,2008-Australian Grand Prix
6,18,7,5,14,17,7,7,2.0,4,55,...,8,1:29.534,213.224,5,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,Australian Grand Prix,2008,2008-Australian Grand Prix
7,18,8,6,1,15,8,8,1.0,3,53,...,4,1:27.903,217.180,5,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Australian Grand Prix,2008,2008-Australian Grand Prix
8,18,9,2,4,2,\N,9,0.0,2,47,...,9,1:28.753,215.100,4,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,Australian Grand Prix,2008,2008-Australian Grand Prix
9,18,10,7,12,18,\N,10,0.0,1,43,...,13,1:29.558,213.166,3,Toyota,Japanese,http://en.wikipedia.org/wiki/Toyota_Racing,Australian Grand Prix,2008,2008-Australian Grand Prix


## Calculate winningest teams with standardized scores

In [57]:
# Group by 'Category', sum 'Value', and sort
StdPoints = rc_df.groupby('ConstructorNME')['StandardPoints'].sum().sort_values(ascending=False)
print(StdPoints)

ConstructorNME
Ferrari      11568
McLaren       7607
Williams      4834
Red Bull      4072
Mercedes      3949
             ...  
MBM              0
Connew           0
Politoys         0
LEC              0
Euro Brun        0
Name: StandardPoints, Length: 211, dtype: int64


In [58]:
fileNme = 'StandardizedPoints.xlsx'
df = StdPoints
export_DfExcel (df, fileNme)

Exported


In [59]:
fileNme = 'RePointed.xlsx'
df = rc_df
export_DfExcel (df, fileNme)

Exported
